<a href="https://colab.research.google.com/github/BenzterBit/MLP/blob/master/AnimeRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"benzybit","key":"2d9bccaa7d4d615e650a0b725f35bb7e"}'}

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d CooperUnion/anime-recommendations-database -p/content
!unzip \*.zip

 20% 5.00M/25.2M [00:00<00:00, 39.5MB/s]
100% 25.2M/25.2M [00:00<00:00, 123MB/s] 
Archive:  anime-recommendations-database.zip
  inflating: anime.csv               
  inflating: rating.csv              


In [5]:

import pandas as pd
import numpy as np
anime_df = pd.read_csv('anime.csv')
for index, row in anime_df.iterrows():
  if row['rating']<7.8:
    anime_df =anime_df.drop([index], axis = 0)
for index, row in anime_df.iterrows():
  if row['type']!='TV':
    anime_df =anime_df.drop([index], axis = 0)
for index, row in anime_df.iterrows():
  if row['members']<=100000:
    anime_df =anime_df.drop([index], axis = 0)
    
rating_df = pd.read_csv('rating.csv')
#rating_df.head(20)
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855


In [6]:
anime_df['genre'] = anime_df.genre.str.split(',')
anime_df = anime_df.drop(['type','rating','episodes','members'],axis=1)
anime_df = anime_df.dropna()
anime_df.head()

,anime_id,name,genre
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,..."
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samur..."
3,9253,Steins;Gate,"[Sci-Fi, Thriller]"
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur..."
6,11061,Hunter x Hunter (2011),"[Action, Adventure, Shounen, Super Power]"


In [7]:
anime_genre_df = anime_df.copy()

for index, row in anime_df.iterrows():
  for gen in row['genre']:
    anime_genre_df.at[index,gen]=1
anime_genre_df = anime_genre_df.fillna(0)
anime_genre_df.head()

,anime_id,name,genre,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Super Power,Drama,Romance,Slice of Life,Supernatural,Mecha,Comedy,School,Sports,Music,Space,Seinen,Mystery,Vampire,Adventure,Horror,Police,Psychological,Mystery,Shoujo,Ecchi,Josei,Martial Arts,Josei,Ecchi,Game,Demons,Harem,Dementia,Psychological,Romance,Horror,Shounen Ai,Dementia,Kids,Fantasy
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samur...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,11061,Hunter x Hunter (2011),"[Action, Adventure, Shounen, Super Power]",1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
files.upload()


Saving Book1.csv to Book1.csv


{'Book1.csv': b'\xef\xbb\xbfuser_id,user_name,user_export_type,user_total_anime,user_total_watching,user_total_completed,user_total_onhold,user_total_dropped,user_total_plantowatch,series_animedb_id,series_title,series_type,series_episodes,my_id,my_watched_episodes,my_start_date,my_finish_date,my_rated,my_score,my_dvd,my_storage,my_status,my_comments,my_times_watched,my_rewatch_value,my_tags,my_rewatching,my_rewatching_ep,update_on_import\r\n5987912,blackrosedragon2,1,342,16,161,8,84,73,31646,3-gatsu no Lion,TV,22,0,9,0000-00-00,0000-00-00,,7,,,Dropped,,0,,,0,0,0\r\n5987912,blackrosedragon2,1,342,16,161,8,84,73,38101,5-toubun no Hanayome,TV,12,0,12,0000-00-00,2019-03-29,,8,,,Completed,,0,,,0,0,0\r\n5987912,blackrosedragon2,1,342,16,161,8,84,73,39783,5-toubun no Hanayome 2nd Season,TV,0,0,0,0000-00-00,0000-00-00,,0,,,Plan to Watch,,0,,,0,0,0\r\n5987912,blackrosedragon2,1,342,16,161,8,84,73,32998,91 Days,TV,12,0,8,0000-00-00,0000-00-00,,6,,,Dropped,,0,,,0,0,0\r\n5987912,blackrosedragon2,

In [12]:
userInput = pd.read_csv('Book1.csv')
userInput = userInput[['series_title','my_score']]
userInput = userInput.rename(index=str , columns = {"series_title":"name","my_score":"rating"})
userInput = userInput.reset_index(drop=True)
input_movies = userInput
input_movies.head()

,name,rating
0,3-gatsu no Lion,7
1,5-toubun no Hanayome,8
2,5-toubun no Hanayome 2nd Season,0
3,91 Days,6
4,Accel World,3


In [13]:
input_id = anime_df[anime_df['name'].isin(input_movies['name'])]
input_id = input_id.drop('genre',1)
input_movies = pd.merge(input_id,input_movies)
input_movies

,anime_id,name,rating
0,5114,Fullmetal Alchemist: Brotherhood,10
1,28977,Gintama°,8
2,9253,Steins;Gate,10
3,11061,Hunter x Hunter (2011),9
4,4181,Clannad: After Story,9
5,918,Gintama,9
6,2904,Code Geass: Hangyaku no Lelouch R2,10
7,28891,Haikyuu!! Second Season,7
8,23273,Shigatsu wa Kimi no Uso,9
9,1575,Code Geass: Hangyaku no Lelouch,9


In [14]:
user_anime = anime_genre_df[anime_genre_df['anime_id'].isin(input_movies['anime_id'])]
user_anime

,anime_id,name,genre,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Super Power,Drama,Romance,Slice of Life,Supernatural,Mecha,Comedy,School,Sports,Music,Space,Seinen,Mystery,Vampire,Adventure,Horror,Police,Psychological,Mystery,Shoujo,Ecchi,Josei,Martial Arts,Josei,Ecchi,Game,Demons,Harem,Dementia,Psychological,Romance,Horror,Shounen Ai,Dementia,Kids,Fantasy
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samur...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,11061,Hunter x Hunter (2011),"[Action, Adventure, Shounen, Super Power]",1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,4181,Clannad: After Story,"[Drama, Fantasy, Romance, Slice of Life, S...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,918,Gintama,"[Action, Comedy, Historical, Parody, Samur...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,2904,Code Geass: Hangyaku no Lelouch R2,"[Action, Drama, Mecha, Military, Sci-Fi, ...",1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,28891,Haikyuu!! Second Season,"[Comedy, Drama, School, Shounen, Sports]",0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,23273,Shigatsu wa Kimi no Uso,"[Drama, Music, Romance, School, Shounen]",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,1575,Code Geass: Hangyaku no Lelouch,"[Action, Mecha, Military, School, Sci-Fi, ...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
user_anime = user_anime.reset_index(drop=True)
user_genre_table =user_anime.drop(['anime_id','name','genre'],1)
user_genre_table

,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Super Power,Drama,Romance,Slice of Life,Supernatural,Mecha,Comedy,School,Sports,Music,Space,Seinen,Mystery,Vampire,Adventure,Horror,Police,Psychological,Mystery,Shoujo,Ecchi,Josei,Martial Arts,Josei,Ecchi,Game,Demons,Harem,Dementia,Psychological,Romance,Horror,Shounen Ai,Dementia,Kids,Fantasy
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
user_profile = user_genre_table.T.dot(input_movies['rating'])
user_profile

Action            319.0
 Adventure        131.0
 Drama            184.0
 Fantasy          182.0
 Magic             27.0
 Military          38.0
 Shounen          295.0
 Comedy           186.0
 Historical        58.0
 Parody            35.0
 Samurai           26.0
 Sci-Fi            93.0
Sci-Fi             10.0
 Thriller          58.0
 Super Power      131.0
Drama              74.0
 Romance          183.0
 Slice of Life    123.0
 Supernatural     248.0
 Mecha             26.0
Comedy            227.0
 School           212.0
 Sports            57.0
 Music              9.0
 Space              8.0
 Seinen            81.0
 Mystery          117.0
 Vampire           25.0
Adventure          42.0
 Horror            32.0
 Police            26.0
 Psychological     82.0
Mystery            36.0
 Shoujo            16.0
Ecchi               6.0
Josei               9.0
 Martial Arts      48.0
 Josei              9.0
 Ecchi             25.0
 Game              40.0
 Demons            21.0
 Harem          

In [17]:
genre_table = anime_genre_df.set_index(anime_genre_df['anime_id'])
genre_table = genre_table.drop(['anime_id','name','genre'],1)
genre_table.head()

,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Super Power,Drama,Romance,Slice of Life,Supernatural,Mecha,Comedy,School,Sports,Music,Space,Seinen,Mystery,Vampire,Adventure,Horror,Police,Psychological,Mystery,Shoujo,Ecchi,Josei,Martial Arts,Josei,Ecchi,Game,Demons,Harem,Dementia,Psychological,Romance,Horror,Shounen Ai,Dementia,Kids,Fantasy
anime_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5114,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28977,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9969,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11061,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
recom_table_df = ((genre_table*user_profile).sum(axis=1)/user_profile.sum())
recom_table_df.head(20)

anime_id
5114     0.326757
28977    0.281189
9253     0.018894
9969     0.281189
11061    0.243401
4181     0.225063
918      0.281189
2904     0.219783
28891    0.270909
23273    0.214782
1575     0.227563
263      0.212003
1        0.255904
30276    0.303695
17074    0.222284
21939    0.236455
457      0.236455
2001     0.209780
245      0.289247
11741    0.224229
dtype: float64

In [19]:
recom_table_sort_df = recom_table_df.sort_values(ascending=False)
recom_table_sort_df.head(10)

anime_id
249      0.442345
21       0.396777
33       0.394276
154      0.378716
121      0.378438
3588     0.378161
1818     0.368714
4898     0.363712
28677    0.360100
6033     0.358989
dtype: float64

In [20]:
anime_df.loc[anime_df['anime_id'].isin(recom_table_sort_df.head(10).keys())]


,anime_id,name,genre
74,21,One Piece,"[Action, Adventure, Comedy, Drama, Fantasy..."
151,33,Berserk,"[Action, Adventure, Demons, Drama, Fantasy..."
200,121,Fullmetal Alchemist,"[Action, Adventure, Comedy, Drama, Fantasy..."
440,3588,Soul Eater,"[Action, Adventure, Comedy, Fantasy, Shoun..."
458,4898,Kuroshitsuji,"[Action, Comedy, Demons, Fantasy, Historic..."
588,6033,Dragon Ball Kai,"[Action, Adventure, Comedy, Fantasy, Marti..."
644,1818,Claymore,"[Action, Adventure, Demons, Fantasy, Shoun..."
668,28677,Yamada-kun to 7-nin no Majo (TV),"[Comedy, Harem, Mystery, Romance, School, ..."
690,249,InuYasha,"[Action, Adventure, Comedy, Demons, Fantas..."
803,154,Shaman King,"[Action, Adventure, Comedy, Drama, Shounen..."


In [0]:
#Upgrades

